In [107]:
pip install -U altair

Note: you may need to restart the kernel to use updated packages.


# Group project report: Pulsar Star Data

### Introduction

In this project, data used to describe **puls**ating **ra**dio **s**ources – also known as **pulsars** – is being investigated.

A pulsar is a rare type of neutron star which in itself is the scientific term for the collapsed core of a massive supergiant star. While rotating, it emits beams of electromagnetic radiation that produce a characteristic pattern of radio emission. Using large radio telescopes, these periodic signals can be detected on earth. However in practice, the majority of detections are caused by radio frequency interference (RFI) and noise, making legitimate signals hard to find.

The HTRU2 dataset (Lyon, 2017) describes a sample of pulsar candidates collected during the High Time Resolution Universe Survey. It contains a total of 17,898 total entries, 16,259 of which being spurious examples caused by RFI/noise, and 1,639 being real pulsar examples. It contains nine variables for each observation. The first four describe statistical characteristics from the integrated pulse profile. This is a version of the recorded signal that has been averaged in both time and frequency. The next four are obtained from the so called DM-SNR curve. This curve shows the spectral signal to noise ratio (SNR) as a function of different dispersion measures (DM). The last variable is a class variable describing whether a pulsar exists or not. 

Pulsars are of considerable scientific interest as probes of space-time, the inter-stellar medium, and states of matter. For that reason it is of high importance to accurately predict whether a type of radio signal observed on earth is a real pulsar or a result of RFI/noise. That will be the main goal of this project.

### Methods & Results

#### Data Cleaning & Wrangling

To analyze a data set accurately, it is crucial to first observe and wrangle the data to prevent formatting issues or null values. This helps choose the best analysis method for the data. First, the required packages are imported.

In [108]:
import pandas as pd
import altair as alt
import numpy as np
from sklearn import set_config
from sklearn.model_selection import train_test_split # importing necessary libraries
np.random.seed(9) # set seed for reproducibility

In [109]:
set_config(transform_output="pandas") # set output as dataframes instead of arrays

The dataset is downloaded from the web (Lyon, 2017) and the file is read using the pandas function read_csv. The first few values of the dataset are shown below:

In [111]:
htru2='https://raw.githubusercontent.com/dorni12/DSCI100_GroupProject/main/HTRU_2.csv'
pulsar= pd.read_csv(htru2,names=[1,2,3,4,5,6,7,8,9],index_col=False) # reading dataset from data file
pulsar.head()

,1,2,3,4,5,6,7,8,9
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0


The data is organized but lacks clear variable names. Thus we used the rename function to change the column names. We also adjusted the values in the 'type' column to be information that is easier to understand. The information initially being displayed as zeros and ones now correspond to the values 'others' and 'pulsar'. (See Table 1)

In [112]:
# renaming column names to meaningful names
pulsar=pulsar.rename(columns={
    1:'mean_IP', # Mean of the integrated profile.
    2:'SD_IP', # Standard deviation of the integrated profile.
    3:'EK_IP', # Excess kurtosis of the integrated profile.
    4:'S_IP', # Skewness of the integrated profile.
    5:'mean_DM-SNR', # Mean of the DM-SNR curve.
    6:'SD_DM-SNR', # Standard deviation of the DM-SNR curve.
    7:'EK_DM-SNR',# Excess kurtosis of the DM-SNR curve.
    8:'S_DM-SNR', # Skewness of the DM-SNR curve.
    9:'type'}) # type of star (others or pulsar)
pulsar['type']=pulsar['type'].replace({
    0:'others',
    1:'pulsar'}) # replacing values of type to more meaningful values

##### Table 1

In [113]:
pulsar.head(5)

,mean_IP,SD_IP,EK_IP,S_IP,mean_DM-SNR,SD_DM-SNR,EK_DM-SNR,S_DM-SNR,type
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,others
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,others
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,others
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,others
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,others


The data frame is then split into training and testing data sets. This allows for accuracy testing in the future.

In [114]:
pulsar_train, pulsar_test = train_test_split(
    pulsar, train_size=0.75, stratify=pulsar["type"]
) # splitting testing and training data

#### Data Analysis

To work with this data, we need to know its basic information. We display it using the info() function (See List 1).

##### List 1

In [115]:
pulsar_train.info() # basic information about training data

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13423 entries, 16444 to 5995
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   mean_IP      13423 non-null  float64
 1   SD_IP        13423 non-null  float64
 2   EK_IP        13423 non-null  float64
 3   S_IP         13423 non-null  float64
 4   mean_DM-SNR  13423 non-null  float64
 5   SD_DM-SNR    13423 non-null  float64
 6   EK_DM-SNR    13423 non-null  float64
 7   S_DM-SNR     13423 non-null  float64
 8   type         13423 non-null  object 
dtypes: float64(8), object(1)
memory usage: 1.0+ MB


We see that all variables are of type 'float64', except for the renamed 'type' column. To further check if there are any null values, the sum of all the null values in each column is calculated (See List 2).

##### List 2

In [116]:
count_nan = pulsar_train.isnull().sum() # total number of null values in each column 
count_nan 

mean_IP        0
SD_IP          0
EK_IP          0
S_IP           0
mean_DM-SNR    0
SD_DM-SNR      0
EK_DM-SNR      0
S_DM-SNR       0
type           0
dtype: int64

There are no null values, so there is no need to drop them.

Next, we calculate all the column's means to distinguish between typical values for pulsars and other observations. We do so by using the 'groupby' and 'mean' functions (See Table 2).

##### Table 2

In [117]:
mean_value=pulsar_train.groupby('type').mean() # mean values of each column for pulsars and other sources
mean_value

,mean_IP,SD_IP,EK_IP,S_IP,mean_DM-SNR,SD_DM-SNR,EK_DM-SNR,S_DM-SNR
type,,,,,,,,
others,116.522794,47.347335,0.211515,0.379930,8.811016,23.224048,8.884452,114.392824
pulsar,56.442077,38.546959,3.146628,15.671984,49.224379,56.232506,2.818744,18.795908


The results show a significant difference in the mean values of all variables for observations of real pulsars and other sources, thus indicating distinctive characteristics between the two classes.

Another example of this is the following graph displaying the correlation between the mean of the integrated profile and its skewness, colored by observations of pulsars and other sources. The graph shows clear separation between pulsars and other sources with some overlap in the middle KNN predictions using KNN classification are expected to be challenging (See Graph 1).

##### Graph 1

In [118]:
alt.data_transformers.disable_max_rows()
pulsar_mean_plot=alt.Chart(pulsar_train,title='Mean verses Skewness of the Integrated Profile (IP)').mark_point(opacity=0.2).encode(
    x=alt.X('mean_IP').title('Mean of IP'),
    y=alt.Y('S_IP').title('Skewness of IP'),
    color=alt.Color('type').title('Type')
)
pulsar_mean_plot 

alt.Chart(...)

Our next step is to compare the amount of total pulsar and other observations in the data set to avoid oversampling due to unequal sample sizes (See List 3).

##### List 3

In [119]:
count_obs = pulsar_train.groupby('type')['type'].count()  # total number of pulsar observations and other observations
count_obs 

type
others    12194
pulsar     1229
Name: type, dtype: int64

Most observations in the data set are of origins other than pulsars. This showcases that pulsars are rare. Resampling of pulsar observations during model training is necessary. First, all the training data is upsampled to account for the rareness of pulsars and prevent undersampling (List 4).

In [120]:
from sklearn.utils import resample
np.random.seed(1)
type_pulsar=pulsar_train[pulsar_train['type']=='pulsar']
type_others=pulsar_train[pulsar_train['type']=='others']
type_others
type_pulsar_upsampled = resample(
    type_pulsar, n_samples=type_others.shape[0],random_state=1
)

upsampled_pulsar = pd.concat((type_pulsar_upsampled ,type_others))

##### List 4

In [121]:
count_obs = upsampled_pulsar.groupby('type')['type'].count()  # total number of pulsar observations and other star observations
count_obs 

type
others    12194
pulsar    12194
Name: type, dtype: int64

Now pulsars and other stars have equal amounts of samples.

In the following, the training data is separated into two parts. One data set is using the values of the Integrated Profile (IP), the other one is based on the data from the DM-SNR curve.

We preprocess the data to later use it for KNN classification by using the 'make_column_transformer' function. We scale the data and afterwards fit the preprocessor module with the matching data set. This process is performed twice to account for the two different kinds of information (See Tables 3 and 4).

##### Table 3

In [123]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer

pulsar_training_IP = upsampled_pulsar[['mean_IP','SD_IP','EK_IP','S_IP','type']]
IP_preprocessor = make_column_transformer(
    (StandardScaler(), ['mean_IP','SD_IP','EK_IP','S_IP']),
    verbose_feature_names_out=False
)
IP_preprocessor.fit(pulsar_training_IP)
scaled_training_IP = IP_preprocessor.transform(pulsar_training_IP)
scaled_training_IP.head()

,mean_IP,SD_IP,EK_IP,S_IP
7648,-1.957970,-0.926013,1.956658,1.904012
12258,-0.103386,0.441090,-0.081545,-0.443850
5706,-1.837102,-1.554636,2.308629,2.662439
1376,-1.311257,-1.161490,1.751848,1.678098
4012,-0.972665,-1.610771,1.369176,1.484640


##### Table 4

In [124]:
pulsar_training_DMSNR= upsampled_pulsar[['mean_DM-SNR','SD_DM-SNR','EK_DM-SNR','S_DM-SNR','type']]
DMSNR_preprocessor = make_column_transformer(
    (StandardScaler(), ['mean_DM-SNR','SD_DM-SNR','EK_DM-SNR','S_DM-SNR']),
    verbose_feature_names_out=False
)
DMSNR_preprocessor.fit(pulsar_training_DMSNR)
scaled_training_DMSNR = DMSNR_preprocessor.transform(pulsar_training_DMSNR)
scaled_training_DMSNR.head()

,mean_DM-SNR,SD_DM-SNR,EK_DM-SNR,S_DM-SNR
7648,1.606828,0.503091,-1.013072,-0.670502
12258,3.023051,1.572052,-1.363560,-0.684306
5706,1.308003,0.669340,-0.987101,-0.665195
1376,0.718420,1.199512,-0.989124,-0.675622
4012,2.007124,2.327763,-1.232032,-0.696430


Now, we want to find the most appropriate value for the number of neighbors (K) to perform KNN classification with both models. First, we set up the pipeline for the IP data set and we define a parameter grid for values of K ranging from 1 to 30.

In [125]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline

IP_knn = KNeighborsClassifier()
IP_tune_pipe = make_pipeline(IP_preprocessor, IP_knn)

In [126]:
IP_parameter_grid = {
    "kneighborsclassifier__n_neighbors": range(1,31,1),
}

Then, we perform cross-validation using 10 folds. We calculate the estimated accuracy for each number of neighbors and plot it in Graph 2.

##### Graph 2

In [127]:
from sklearn.model_selection import GridSearchCV

IP_tune_grid = GridSearchCV(
    estimator=IP_tune_pipe,
    param_grid=IP_parameter_grid,
    cv=10
)

IP_accuracies_grid = pd.DataFrame(
    IP_tune_grid.fit(
        scaled_training_IP,
        upsampled_pulsar["type"]
    ).cv_results_
)

cross_val_plot=alt.Chart(IP_accuracies_grid).mark_line(point=True).encode(
    y=alt.Y("mean_test_score").title('Mean Estimated Accuracy').scale(zero=False),
    x=alt.X("param_kneighborsclassifier__n_neighbors").title('Number of Neighbors (K)'),
)
cross_val_plot

alt.Chart(...)

From this graph, we see that the highest test score is reached when K=1. However, this would lead to over-fitting and thus would provide less useful data. K=16 could be a useful value as it reaches a high test score and the scores for higher and lower values of K don't vary too much: Also, it does not require a significant amount of computational power.

The same process described above is repeated for the data from the DM-SNR curve. The results are shown in Graph 3.

In [128]:
DMSNR_knn = KNeighborsClassifier()
DMSNR_tune_pipe = make_pipeline(DMSNR_preprocessor, DMSNR_knn)

In [129]:
DMSNR_parameter_grid = {
    "kneighborsclassifier__n_neighbors": range(1,31,1),
}

##### Graph 3

In [130]:
DMSNR_tune_grid = GridSearchCV(
    estimator=DMSNR_tune_pipe,
    param_grid=DMSNR_parameter_grid,
    cv=10
)

DMSNR_accuracies_grid = pd.DataFrame(
    DMSNR_tune_grid.fit(
        scaled_training_DMSNR,
        upsampled_pulsar["type"]
    ).cv_results_
)

cross_val_plot=alt.Chart(DMSNR_accuracies_grid).mark_line(point=True).encode(
    y=alt.Y("mean_test_score").title('Mean Estimated Accuracy').scale(zero=False),
    x=alt.X("param_kneighborsclassifier__n_neighbors").title('Number of Neighbors (K)'),
)
cross_val_plot

alt.Chart(...)

According to the graph, the highest score is again K=1. However as mentioned before, this would lead to over-fitting. Therefore, other values of K are considered. Between the values of K=6, K=11, and K=16, the difference between the scores for nearby values is quite large. We chose the final value to be K=21. It has a moderate score of around 88% estimated accuracy, and the difference between the nearby values is less than 1%. Furthermore, it does not require a significant amount of computational power.

Now, the optimal values for K are used to fit both the IP and the DM-SNR models.

In [131]:
IP_knn = KNeighborsClassifier(n_neighbors=16) 
X = scaled_training_IP[['mean_IP','SD_IP','EK_IP','S_IP']]
y = upsampled_pulsar ["type"]
IP_fit = make_pipeline(IP_preprocessor, IP_knn).fit(X, y)

In [132]:
DMSNR_knn = KNeighborsClassifier(n_neighbors=21) 
X = scaled_training_DMSNR[['mean_DM-SNR','SD_DM-SNR','EK_DM-SNR','S_DM-SNR']]
y = upsampled_pulsar ["type"]
DMSNR_fit = make_pipeline(DMSNR_preprocessor, DMSNR_knn).fit(X, y)

For further analysis, we create a new data frame containing only the IP data as well as its real type. We assign a new column containing the type that was predicted with our model. Table 5 shows a few instances where the prediction did not match the true value.

In [75]:
pulsar_IP = pulsar[["mean_IP", "SD_IP", "EK_IP", "S_IP", "type"]]

In [76]:
pulsar_IP = pulsar_IP.assign(predicted_type = IP_fit.predict(pulsar[["mean_IP", "SD_IP", "EK_IP", "S_IP"]]))

##### Table 5

In [133]:
pulsar_IP[pulsar_IP["type"] != pulsar_IP["predicted_type"]].head()

,mean_IP,SD_IP,EK_IP,S_IP,type,predicted_type
19,99.367188,41.572202,1.547197,4.154106,pulsar,others
42,120.554688,45.549905,0.282924,0.419909,pulsar,others
61,27.765625,28.666042,5.770087,37.419009,pulsar,others
92,23.625000,29.948654,5.688038,35.987172,pulsar,others
93,94.585938,35.779823,1.187309,3.687469,pulsar,others


Now, we calculate the actual accuracy of the IP model and display our results in a confusion matrix shown in Table 6.

In [134]:
## Finding the Accuracy of the model
correct_preds = pulsar_IP[
    pulsar_IP['type'] == pulsar_IP['predicted_type']
]

correct_preds.shape[0] / pulsar_IP.shape[0]

0.9123365739188736

##### Table 6

In [135]:
## Making Confusion Matrix
confusion_matrix_IP = pd.crosstab(
    pulsar_IP["type"],
    pulsar_IP["predicted_type"]
)
confusion_matrix_IP

predicted_type,others,pulsar
type,,
others,16259,0
pulsar,1569,70


We repeat the same procedure for the data from the DM-SNR curve. The results are shown in Tables 7 and 8.

In [136]:
pulsar_SNR = pulsar[["mean_DM-SNR", "SD_DM-SNR", "EK_DM-SNR", "S_DM-SNR", "type"]]

In [137]:
pulsar_SNR = pulsar_SNR.assign(predicted_type = DMSNR_fit.predict(pulsar[["mean_DM-SNR", "SD_DM-SNR", "EK_DM-SNR", "S_DM-SNR", "type"]]))

##### Table 7

In [138]:
pulsar_SNR[pulsar_SNR["type"] != pulsar_SNR["predicted_type"]].head()

,mean_DM-SNR,SD_DM-SNR,EK_DM-SNR,S_DM-SNR,type,predicted_type
19,27.555184,61.719016,2.208808,3.662680,pulsar,others
42,1.358696,13.079034,13.312141,212.597029,pulsar,others
61,73.112876,62.070220,1.268206,1.082920,pulsar,others
92,146.568562,82.394624,-0.274902,-1.121848,pulsar,others
93,6.071070,29.760400,5.318767,28.698048,pulsar,others


In [139]:
## Finding the Accuracy of the model
correct_preds_SNR = pulsar_SNR[
    pulsar_SNR['type'] == pulsar_SNR['predicted_type']
]

correct_preds_SNR.shape[0] / pulsar_SNR.shape[0]

0.908425522404738

##### Table 8

In [140]:
## Making Confusion Matrix
confusion_matrix_SNR = pd.crosstab(
    pulsar_SNR["type"],
    pulsar_SNR["predicted_type"]
)
confusion_matrix_SNR

predicted_type,others,pulsar
type,,
others,16096,163
pulsar,1476,163


By comparing the results of the IP and the DM-SNR models, we can see that the IP model reaches a higher accuracy score. That is why we would recommend using the IP model to predict, if a new observation made can be considered a pulsar or not.

### Discussion

Based on our findings, we can conclude that the KNN classification model that uses data from the Integrated Profile (IP) seems to be more accurate than the one that uses data from the DM-SNR curve. Thus, we recommend using the IP model to differentiate pulsars from RFI/noise.

This is a result we are happy with as we were looking for a reliable way to differentiate between real pulsars and RFI/noise using existing measuring data. Ending up with a model that predicts pulsars with an accuracy of 91.2% can be considered a success.

Pulsars are a rare type of neutron stars and it is scientifically of high importance to being able to precisely separate them from other phenomena. The successful identification of genuine pulsars could have a huge impact on astrological researches as this has the potential to help us improve our understanding of them and their characteristics like structure, temperature, or gravitational condition.

Moreover, approximately 24 gamma-ray pulsars are continuously discovered in the *Fermi* Large Area Telescope (LAT) every year. The accurate prediction model could play a crucial part to efficiently identify potential sources of gamma-ray pulsations as well. Furthermore, there are diverse discovery channels such as deep radio searches, blind gamma-ray periodicity searches, and phase-folding gamma-rays that have observed and discovered known pulsars. The prediction model could identify which pulsars are discovered from which channel using each channel's characteristics (Smith et al., 2019).

The identification of real pulsar observations using classification could lead to more advanced models for pulsar detection and analysis in the future including potentially integrating artificial intelligence to enhance accuracy. Furthermore, by differentiating real pulsars from other phenomena better than before, this model could help improve the understanding of pulsar emissions across different wavelengths (Johnson el al., 2013).

### References

Lyon, Robert. (2017). HTRU2. UCI Machine Learning Repository. https://doi.org/10.24432/C5DK6R.

Johnson, T.J., Guillemot, L., Kerr, M., et al. (2013). BROADBAND PULSATIONS FROM PSR B1821−24:IMPLICATIONS FOR EMISSION MODELS AND THE PULSAR POPULATION OF M28. The Astrophysical Journal, 778(106), 8-10. https://doi.org/10.1088/0004-637X/778/2/106.

Smith, D.A., Bruel, P., Cognard, I., et al. (2019). Searching a Thousand Radio Pulsars for Gamma-Ray Emission. The Astrophysical Journal, 871(78), 5-8. https://doi.org/10.3847/1538-4357/aaf57d.